In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import tweepy
from tweepy import OAuthHandler
import re

In [3]:
#filename = "wonderland.txt"
#raw_text = open(filename).read()
#raw_text = raw_text.lower()


consumer_key = "1vnx8rybX0CvnERfLOfZRkJmN"
consumer_secret = 'pYSzrmJqlLT3A8znLBpSzt3djC0NwGAIDctLeJevHOIX9KBlfM'
access_token = '1478601054-yjTbHT3FJdLCleRbZnnmFcsqkIroSKeKqZROCPw'
access_secret = 'cHKGTKWcEds0q6FjdwJPV01qt6L2bhvFTuQAxWtdDHHFj'
        
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

tweets = ""
for i in range(5):
    new_tweets = api.user_timeline(screen_name = 'wojespn',count=200)
    for tweet in new_tweets:
        tweets += re.sub(r'http\S+', '', tweet.text)
        tweets += "\n"
raw_text = tweets.lower()


In [5]:

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)

['\n', ' ', '!', '"', '#', '$', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’', '…', '\u2066', '\u2069']


In [27]:
outFile = open("corpus.txt", "w")
outFile.write(raw_text)

101460

In [4]:
inFile = open("corpus.txt", "r")
lines = [line for line in inFile.readlines()]
#print(lines)
raw_text = "".join(lines)


In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  101460
Total Vocab:  58


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 75
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  101385


In [8]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [9]:
# define the LSTM model
model = Sequential()
print(X.shape)
model.add(LSTM(256, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(256,)))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

(101385, 75, 1)


In [51]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [53]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
101385/101385 [==============================] - 583s 6ms/step - loss: 3.0906
Epoch 2/20
101385/101385 [==============================] - 580s 6ms/step - loss: 2.8245
Epoch 3/20
101385/101385 [==============================] - 580s 6ms/step - loss: 2.5287
Epoch 4/20
101385/101385 [==============================] - 579s 6ms/step - loss: 2.2967
Epoch 5/20
101385/101385 [==============================] - 581s 6ms/step - loss: 2.0849
Epoch 6/20
101385/101385 [==============================] - 581s 6ms/step - loss: 1.8857
Epoch 7/20
101385/101385 [==============================] - 582s 6ms/step - loss: 1.6919
Epoch 8/20
101385/101385 [==============================] - 581s 6ms/step - loss: 1.5127
Epoch 9/20
101385/101385 [==============================] - 580s 6ms/step - loss: 1.3557
Epoch 10/20
101385/101385 [==============================] - 622s 6ms/step - loss: 1.2095
Epoch 11/20
101385/101385 [==============================] - 588s 6ms/step - loss: 1.0837
Epoch 12/20
101385/

In [10]:
filename = "weights-improvement-20-0.4724.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
result

NameError: name 'result' is not defined

In [15]:
import sys
# pick a random seed
for i in range(5):
    start = numpy.random.randint(0, len(dataX)-1)
    pattern = dataX[start]
    print(pattern)
    print("Seed:")
    print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    print("end seed")
    # generate characters
    for i in range(140):
        count = 0
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        
        #if result == "\n":
            
        #    break
         
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)

        pattern = pattern[1:len(pattern)]
    print("\nDone.")

[36, 39, 39, 1, 30, 42, 41, 46, 36, 31, 32, 45, 1, 48, 43, 42, 41, 1, 30, 39, 32, 28, 45, 36, 41, 34, 1, 50, 28, 36, 49, 32, 45, 46, 9, 1, 39, 32, 28, 34, 48, 32, 1, 46, 42, 48, 55, 1, 0, 37, 28, 53, 53, 1, 50, 28, 36, 49, 32, 31, 1, 37, 42, 41, 28, 46, 1, 37, 32, 45, 32, 29, 38, 42, 11]
Seed:
" ill consider upon clearing waivers, league sou… 
jazz waived jonas jerebko. "
end seed

espn story on fusuf uurki cgreec to a two-year, $12m deal. 
free agent guard tyreke evans is nearing a one-year deal with the indiana pacer
Done.
[39, 42, 47, 47, 32, 9, 1, 39, 32, 28, 34, 48, 32, 1, 46, 42, 48, 45, 30, 32, 1, 47, 32, 39, 39, 46, 1, 32, 46, 43, 41, 0, 32, 46, 43, 41, 1, 46, 42, 48, 45, 30, 32, 46, 1, 50, 36, 47, 35, 1, 26, 45, 42, 52, 30, 32, 52, 42, 48, 41, 34, 23, 1, 42, 38, 39, 28, 35, 42, 40, 28, 1, 30, 36, 47]
Seed:
" lotte, league source tells espn
espn sources with @royceyoung: oklahoma cit "
end seed
y, carmelo anthony will part ways thes wime makens and tas beree boronty'y frens toe